In [17]:
import pygmsh
import numpy as np


# Parameters for alternating rectangles
domain_size = 1.0
wide_height = 0.15
thin_height = 0.05
# domain_width = domain_size
total_height = domain_size
num_layers = int(total_height // (wide_height + thin_height))
domain_width = num_layers * (wide_height + thin_height) + thin_height
mesh_resolution = 0.1  # Desired triangle size

# Generate geometry and mesh
with pygmsh.geo.Geometry() as geom:
    model = geom.__enter__()

    y0 = -thin_height
    y1 = 0
    model.add_rectangle(
            xmin=0.0,
            xmax=domain_width,
            ymin=y0,
            ymax=y1,
            z=0.0,
            mesh_size=mesh_resolution,
        )

    for i in range(num_layers):
        # Wide rectangle
        y0 = i * (wide_height + thin_height)
        y1 = y0 + wide_height
        if y1 > total_height:
            break
        model.add_rectangle(
            xmin=0.0,
            xmax=domain_width,
            ymin=y0,
            ymax=y1,
            z=0.0,
            mesh_size=mesh_resolution,
        )

        # Thin rectangle
        y0 = y1
        y1 = y0 + thin_height
        if y1 > total_height:
            break
        model.add_rectangle(
            xmin=0.0,
            xmax=domain_width,
            ymin=y0,
            ymax=y1,
            z=0.0,
            mesh_size=mesh_resolution,
        )

    mesh = geom.generate_mesh()

# Extract points and triangle faces
points = mesh.points[:, :3]  # retain z=0 for OBJ format
triangles = None
for cell_block in mesh.cells:
    if cell_block.type == "triangle":
        triangles = cell_block.data
        break

# Write to .obj file
with open("alternating_rectangles_mesh.obj", "w") as f:
    # Write vertices
    for v in points:
        f.write(f"v {v[0]} {v[1]} {v[2]}\n")
    # Write triangle faces (OBJ uses 1-based indexing)
    for tri in triangles:
        f.write(f"f {tri[0]+1} {tri[1]+1} {tri[2]+1}\n")

print(f"✅ Mesh exported to 'alternating_rectangles_mesh.obj' with {len(points)} vertices and {len(triangles)} triangles.")

# Tag each triangle in the mesh
triangle_tags = []
for tri in triangles:
    # Calculate the centroid of the triangle
    centroid = points[tri].mean(axis=0)
    # Determine the layer based on the y-coordinate of the centroid
    if centroid[1] < 0:
        triangle_tags.append("0")
    for i in range(num_layers):
        y0 = i * (wide_height + thin_height)
        y1 = y0 + wide_height
        if y0 <= centroid[1] < y1:
            # triangle_tags.append(f"Layer {i + 1} (Wide)")
            triangle_tags.append(f"{2*i + 1}")
            break
        y0 = y1
        y1 = y0 + thin_height
        if y0 <= centroid[1] < y1:
            # triangle_tags.append(f"Layer {i + 1} (Thin)")
            triangle_tags.append(f"{2*i + 2}")
            break

# Write triangle tags to a file
with open("alternating_rectangles_mesh_tags.dat", "w") as tag_file:
    for idx, tag in enumerate(triangle_tags):
        tag_file.write(f"{tag}\n")
        # tag_file.write(f"{tag}\n")

print("✅ Triangle tags exported to 'alternating_rectangles_mesh.dat'.")

# Fuse vertices by rounding to a tolerance and finding unique points
tolerance = 1e-6
rounded_points = np.round(points / tolerance) * tolerance
unique_points, inverse_indices = np.unique(rounded_points, axis=0, return_inverse=True)

# Update the triangle faces to use the new vertex indices
fused_triangles = inverse_indices[triangles]

# Update the triangle tags to match the new fused triangles
fused_triangle_tags = triangle_tags  # Tags remain the same as they are per triangle

# Write the fused mesh to a new .obj file
with open("fused_alternating_rectangles_mesh.obj", "w") as f:
    # Write unique vertices
    for v in unique_points:
        f.write(f"v {v[0]} {v[1]} {v[2]}\n")
    # Write triangle faces (OBJ uses 1-based indexing)
    for tri in fused_triangles:
        f.write(f"f {tri[0]+1} {tri[1]+1} {tri[2]+1}\n")

# Write the updated triangle tags to a new file
with open("fused_alternating_rectangles_mesh_tags.dat", "w") as tag_file:
    for tag in fused_triangle_tags:
        tag_file.write(f"{tag}\n")

print(f"✅ Fused mesh exported to 'fused_alternating_rectangles_mesh.obj' with {len(unique_points)} vertices and {len(fused_triangles)} triangles.")

✅ Mesh exported to 'alternating_rectangles_mesh.obj' with 269 vertices and 332 triangles.
✅ Triangle tags exported to 'alternating_rectangles_mesh.dat'.
✅ Fused mesh exported to 'fused_alternating_rectangles_mesh.obj' with 189 vertices and 332 triangles.
